# Cleaning a PostgreSQL Database
![Clean PostgreSQL Database](Project_Image.jpeg)

In this project, you will work with data from a hypothetical Super Store to challenge and enhance your SQL skills in data cleaning. This project will engage you in identifying top categories based on the highest profit margins and detecting missing values, utilizing your comprehensive knowledge of SQL concepts.

## Data Dictionary:

### `orders`:
| Column | Definition | Data type | Comments |
|--------|------------|-----------|----------|
| `row_id`| Unique Record ID | `INTEGER` |
| `order_id` | Identifier for each order in table | `TEXT` | Connects to `order_id` in `returned_orders` table |
| `order_date` | Date when order was placed | `TEXT` |
| `market` | Market order_id belongs to | `TEXT` |
| `region` | Region Customer belongs to | `TEXT` | Connects to `region` in `people` table |
| `product_id` | Identifier of Product bought | `TEXT` | Connects to `product_id` in `products` table |
| `sales` | Total Sales Amount for the Line Item | `DOUBLE PRECISION` |
| `quantity` | Total Quantity for the Line Item | `DOUBLE PRECISION` |
| `discount` | Discount applied for the Line Item | `DOUBLE PRECISION` |
| `profit` | Total Profit earned on the Line Item | `DOUBLE PRECISION` |

### `returned_orders`:
| Column | Definition | Data type |
|--------|------------|-----------|
| `returned`| Yes values for Order / Line Item Returned | `TEXT` |
| `order_id` | Identifier for each order in table | `TEXT` |
| `market` | Market order_id belongs to | `TEXT` |

### `people`:
| Column | Definition | Data type |
|--------|------------|-----------|
| `person`| Name of Salesperson credited with Order | `TEXT` |
| `region` | Region Salesperson in operating in | `TEXT` |

### `products`:
| Column | Definition | Data type |
|--------|------------|-----------|
| `product_id`| Unique Identifier for the Product | `TEXT` |
| `category` | Category Product belongs to | `TEXT` |
| `sub_category` | Sub Category Product belongs to | `TEXT` |
| `product_name` | Detailed Name of the Product | `TEXT` |

As you can see in the Data Dictionary above, date fields have been written to the `orders` table as `TEXT` and numeric fields like sales, profit, etc. have been written to the `orders` table as `Double Precision`. You will need to take care of these types in some of the queries. This project is an excellent opportunity to apply your SQL skills in a practical setting and gain valuable experience in data cleaning and analysis. Good luck, and happy querying!

In [30]:
-- top_five_products_each_category
WITH product_info AS (
	SELECT p.product_name, p.category,
	ROUND(SUM(o.sales)::numeric, 2) AS product_total_sales, 
	ROUND(SUM(o.profit)::numeric, 2) AS product_total_profit
	FROM products AS p
	LEFT JOIN orders AS o
	ON p.product_id = o.product_id
	GROUP BY p.product_name, p.category
),

	top_products AS(
SELECT 
	RANK() OVER(PARTITION BY category ORDER BY 	
	product_total_sales DESC) AS product_rank, 
	product_name, category,	product_total_sales, 
	product_total_profit
	FROM product_info
)
SELECT *
FROM top_products
WHERE product_rank <= 5;




,product_rank,product_name,category,product_total_sales,product_total_profit
0,1,"Hon Executive Leather Armchair, Adjustable",Furniture,58193.48,5997.25
1,2,"Office Star Executive Leather Armchair, Adjust...",Furniture,51449.80,4925.80
2,3,"Harbour Creations Executive Leather Armchair, ...",Furniture,50121.52,10427.33
3,4,"SAFCO Executive Leather Armchair, Black",Furniture,41923.53,7154.28
4,5,"Novimex Executive Leather Armchair, Adjustable",Furniture,40585.13,5562.35
5,1,"Eldon File Cart, Single Width",Office Supplies,39873.23,5571.26
6,2,"Hoover Stove, White",Office Supplies,32842.60,-2180.63
7,3,"Hoover Stove, Red",Office Supplies,32644.13,11651.68
8,4,"Rogers File Cart, Single Width",Office Supplies,29558.82,2368.82
9,5,"Smead Lockers, Industrial",Office Supplies,28991.66,3630.44


In [31]:
-- impute_missing_values
WITH missing_quanities AS (
	SELECT product_id, discount, market, region, quantity, sales
	FROM orders
	WHERE quantity IS NULL
),

unit_prices AS(
	SELECT product_id, AVG(CAST(sales / quantity AS numeric)) AS 
	unit_price
	FROM orders
	WHERE quantity IS NOT NULL
	GROUP BY product_id
)

SELECT DISTINCT m.*, 
	ROUND(CAST(m.sales AS NUMERIC) / u.unit_price, 0) AS calculated_quantity
FROM missing_quanities AS m
INNER JOIN unit_prices AS u
ON m.product_id = u.product_id


,product_id,discount,market,region,quantity,sales,calculated_quantity
0,FUR-ADV-10000571,0.00,EMEA,EMEA,NaN,438.960,4
1,FUR-ADV-10004395,0.00,EMEA,EMEA,NaN,84.120,3
2,FUR-BO-10001337,0.15,US,West,NaN,308.499,3
3,TEC-STA-10003330,0.00,Africa,Africa,NaN,506.640,2
4,TEC-STA-10004542,0.00,Africa,Africa,NaN,160.320,5
